In [ ]:
import numpy as np
import pandas as pd
import re
from requests_html import HTMLSession

First we download the data for 2017

In [ ]:
cpi2017 = pd.read_excel('http://files.transparency.org/content/download/2172/13704/file/CPI2017_FullDataSet.xlsx', skiprows=2, skipfooter=2)

Then for prior years.

In [ ]:
cpi_older = pd.read_excel('http://www.transparency.org/assets/data/sets/cpi_1995_2016.xlsx',
                         header=1, skiprows=5)

We are only interested in data for years since 2012, where CPI methodology became year-on-year comparable.  We will also retain the World Bank income group and region code.

In [ ]:
cpi_older = cpi_older.loc[1:, [
    'Country/Territory',
    'Income Group based in World Bank 10 definition', 'Region',
    '2016 CPI Score', '2015 CPI Score', '2014 CPI Score', '2013 CPI Score',
    '2012 CPI Score', 
]]
cpi_older.head()

In [ ]:
cpi_older.tail()

To make the datasets joinable, we need to change the country names to ISO 3166-1 alpha-3 codes, as are used in the 2017 data set.

In [ ]:
countries = pd.read_csv('https://pkgstore.datahub.io/core/country-codes/country-codes_csv/data/3b9fd39bdadd7edd7f7dcee708f47e1b/country-codes_csv.csv')[[
    'ISO3166-1-Alpha-2',
    'ISO3166-1-Alpha-3',
    'official_name_en',
    'CLDR display name'
]]

In [ ]:
countries.head()

In [ ]:
codes = {}
for row in countries.itertuples(index=False):
    code2, code3, name, shortname = row
    codes[name] = (code3, code2)
    codes[shortname] = (code3, code2)

In [ ]:
cpi_older = cpi_older.set_index('Country/Territory')

Define the slice data frame

In [ ]:
cpi = pd.DataFrame(columns=['country', 'year', 'cpi']).set_index(['country', 'year'])

Define the dimension dataframes

In [ ]:
countries = pd.DataFrame(columns=['country', 'region', 'incomegroup']).set_index(['country'])

In [ ]:
regions = pd.DataFrame({'region': sorted(set(cpi_older['Region']))})
regions

In [ ]:
incomegroups = pd.DataFrame({'incomegroup': sorted(set(cpi_older['Income Group based in World Bank 10 definition'].dropna()))})
incomegroups

Check whether the country name to ISO 3166-1 codes mapping is complete

In [ ]:
for country in cpi_older.index:
    code = codes.get(country)
    if code is None:
        print(country)

Fix up the mismatches manually.

In [ ]:
codes.update({
    'Congo Republic': ('COG', 'CG'),
    'Côte d´Ivoire': ('CIV', 'CI'),
    'Czech Republic': ('CZE', 'CZ'),
    'The Democratic Republic of Congo': ('COD', 'CD'),
    'The FYR of Macedonia': ('MKD', 'MK'),
    'Korea (North)': ('PRK', 'KP'),
    'Korea (South)': ('KOR', 'KR'),
    'Kosovo': ('KSV', 'XK'),
    'Sao Tome & Principe': ('STP', 'ST'),
    'United Kingdom': ('GBR', 'GB'),
    'United States': ('USA', 'US') ,
    'Yugoslavia': ('YUG', 'YU'),
})

Now we can fill the dimension and CPI slice (for 2012-2016) data frames.

In [ ]:
for country in cpi_older.index:
    code = codes[country][1]
    countries.loc[code, 'region'] = cpi_older.loc[country, 'Region']
    countries.loc[code, 'incomegroup'] = cpi_older.loc[country, 'Income Group based in World Bank 10 definition']
    for year in range(2012, 2017):
        cpi.loc[(code, year), 'cpi'] =  cpi_older.loc[country, str(year) + ' CPI Score']
        

And now add the 2017 data to the CPI slice dataframe -- it needs to be mapped from alpha3 to alpha2 code.

In [ ]:
codes3to2 = dict(codes.values())

In [ ]:
cpi2017['year'] = 2017
cpi2017['ISO2'] = cpi2017['ISO3'].apply(lambda x: codes3to2[x])
cpi2017 = cpi2017[['ISO2', 'year', 'CPI Score 2017']].rename(columns={
    'ISO2': 'country',
    'CPI Score 2017': 'cpi'
}).set_index(['country', 'year'])

In [ ]:
cpi = cpi.reset_index().append(cpi2017['cpi'].reset_index()).dropna().set_index(['country', 'year']).sort_index().dropna()

And write the dataframes out to CSV.

In [ ]:
cpi.to_csv('cpi_by_country.csv')

In [ ]:
countries.sort_index().reset_index().dropna().to_csv('countries.csv', index=False)
regions.sort_values(by='region').to_csv('regions.csv', index=False)
incomegroups.sort_values(by='incomegroup').to_csv('incomegroups.csv', index=False)

Now make some aggregated slices for regions and incomegroups

In [ ]:
cpi = cpi.reset_index()

In [ ]:
cpi['region'] = cpi['country'].apply(lambda x: countries.loc[x, 'region'])
cpi['incomegroup'] = cpi['country'].apply(lambda x: countries.loc[x, 'incomegroup'])

In [ ]:
cpi.groupby(['region', 'year']).agg(np.mean)[['cpi']].to_csv('cpi_by_region.csv')

In [ ]:
cpi.groupby(['incomegroup', 'year']).agg(np.mean)[['cpi']].to_csv('cpi_by_incomegroup.csv')